In [ ]:
import pandas as pd

In [ ]:
import os

#Sept 9

##Setup

In [ ]:
date = '2022-09-09'

In [ ]:
powerAmplitudeDF = pd.read_csv('2022-09-09-Power.csv')
del powerAmplitudeDF['Unnamed: 0']
powerAmplitudeDF.head()

In [ ]:
powerAmplitudeDF.shape

In [ ]:
acousticFeatsDF = pd.read_csv('AcousticFeatures_noModeratorSpeech_aDBS012_' + '2022-09-09_' + 'audio_amplitude.csv')
del acousticFeatsDF['Unnamed: 0']
acousticFeatsDF.head()

In [ ]:
list(acousticFeatsDF.columns)

In [ ]:
acousticFeatsDF.shape

In [ ]:
powerAmplitudeDF = powerAmplitudeDF.iloc[:acousticFeatsDF.shape[0]]
powerAmplitudeDF.shape

In [ ]:
powerAmplitudeAllLLDdf = pd.concat([powerAmplitudeDF,acousticFeatsDF],axis=1)
powerAmplitudeAllLLDdf.shape

##t-test setup

Website to compute p value from r value and n: http://vassarstats.net/tabs_r.html

alternative source: https://www.socscistatistics.com/pvalues/pearsondistribution.aspx

Need r >= 0.077 for p < 0.01
Need r >= 0.059 for p < 0.05

In [ ]:
rValFor99ConfidenceLevel = 0.077
rValFor95ConfidenceLevel = 0.059

In [ ]:
from math import sqrt
from scipy.stats import t

def calculate_p_value(r, n):
    # Calculate t-statistic
    t_statistic = r * sqrt((n - 2) / (1 - r**2))

    # Degrees of freedom
    degrees_of_freedom = n - 2

    # Calculate two-tailed p-value
    p_value = 2 * (1 - t.cdf(abs(t_statistic), degrees_of_freedom))

    return t_statistic, degrees_of_freedom, p_value

# Example usage
correlation_coefficient = 0.059
sample_size = 1136
t_stat, df, p_value = calculate_p_value(correlation_coefficient, sample_size)

print(f"Correlation Coefficient: {correlation_coefficient}")
print(f"T-statistic: {t_stat}")
print(f"Degrees of Freedom: {df}")
print(f"P-value: {p_value}")



Correlation Coefficient: 0.059
T-statistic: 1.9902871871130796
Degrees of Freedom: 1134
P-value: 0.046799066463467875


In [ ]:
t_stat, df, p_value = calculate_p_value(0.058, sample_size)
assert p_value > 0.05

##random noise t-test (10)

In [ ]:
import numpy as np

In [ ]:
# Set a seed for reproducibility (optional)
np.random.seed(33)

# Mean and standard deviation of the Gaussian distribution
mean_value = 0
std_dev = 1

# Generate a random 1136-element numpy array from a Gaussian distribution
random_array = np.random.normal(mean_value, std_dev, 1136)


In [ ]:
powerAmplitudeAllLLDdf['Noise_Seed42'] = random_array

In [ ]:
powerAmplitudeAllLLDdf.corr()['Noise_Seed42'][4:-1]#.abs()

F0final_sma                 -0.064561
voicingFinalUnclipped_sma   -0.030557
jitterLocal_sma             -0.011761
jitterDDP_sma               -0.027644
shimmerLocal_sma            -0.008827
                               ...   
lspFreq_sma_de[7]            0.020479
pcm_zcr_sma_de.1            -0.001696
voiceProb_sma_de            -0.045858
F0_sma_de                   -0.029190
F0env_sma_de                 0.006997
Name: Noise_Seed42, Length: 207, dtype: float64

In [ ]:
correlations_Noise = powerAmplitudeAllLLDdf.corr()['Noise_Seed42'][4:-1]#.abs()

In [ ]:
positiveCorrelationCondition = correlations_Noise > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Noise < -1* rValFor95ConfidenceLevel
noiseCorrelates = correlations_Noise[ positiveCorrelationCondition | negativeCorrelationCondition]
noiseCorrelates.head = 'r'

print("# of acoustic features where p < 0.05:", len(noiseCorrelates))
noiseCorrelates.nlargest(len(noiseCorrelates)) # for sorting

# of acoustic features where p < 0.05: 10


mfcc_sma_de[14]         0.073175
mfcc_sma[4].1           0.067862
mfcc_sma[4]             0.066350
mfcc4_sma3              0.066350
audSpec_Rfilt_sma[9]   -0.059178
audSpec_Rfilt_sma[6]   -0.061485
audSpec_Rfilt_sma[4]   -0.062194
audSpec_Rfilt_sma[5]   -0.062241
F0final_sma            -0.064561
audSpec_Rfilt_sma[8]   -0.066369
Name: Noise_Seed42, dtype: float64

##Right vLPFC

In [ ]:
correlations_Right_vlPFC = powerAmplitudeAllLLDdf.corr()['Power: Right vlPFC'][4:]#.abs()

In [ ]:
positiveCorrelationCondition = correlations_Right_vlPFC > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Right_vlPFC < -1* rValFor95ConfidenceLevel
AcousticCorrelates95_Right_vlPFC = correlations_Right_vlPFC[ positiveCorrelationCondition | negativeCorrelationCondition]
AcousticCorrelates95_Right_vlPFC.head = 'r'

print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Right_vlPFC))
AcousticCorrelates95_Right_vlPFC.nlargest(len(AcousticCorrelates95_Right_vlPFC)) # for sorting

# of acoustic features where p < 0.05: 89


pcm_RMSenergy_sma           0.167584
pcm_loudness_sma            0.164787
Loudness_sma3               0.162632
audspec_lengthL1norm_sma    0.162632
spectralFlux_sma3           0.161759
                              ...   
mfcc_sma[12].1             -0.150060
lspFreq_sma[7]             -0.154565
lspFreq_sma[2]             -0.155497
lspFreq_sma[4]             -0.158998
mfcc_sma[5].1              -0.161376
Name: Power: Right vlPFC, Length: 89, dtype: float64

In [ ]:
AcousticCorrelates95_Right_vlPFC.nlargest(len(AcousticCorrelates95_Right_vlPFC)).to_csv(date + '_noModeratorSpeech' + '_AcousticCorrelates95_Right_vlPFC.csv',header=['r'],index_label='Acoustic Feature')

In [ ]:
print(AcousticCorrelates95_Right_vlPFC.index.tolist())

['F0final_sma', 'voicingFinalUnclipped_sma', 'jitterLocal_sma', 'jitterDDP_sma', 'shimmerLocal_sma', 'logHNR_sma', 'audspec_lengthL1norm_sma', 'pcm_RMSenergy_sma', 'pcm_zcr_sma', 'audSpec_Rfilt_sma[7]', 'audSpec_Rfilt_sma[8]', 'audSpec_Rfilt_sma[9]', 'pcm_fftMag_fband250-650_sma', 'pcm_fftMag_fband1000-4000_sma', 'pcm_fftMag_spectralRollOff25.0_sma', 'pcm_fftMag_spectralRollOff50.0_sma', 'pcm_fftMag_spectralRollOff75.0_sma', 'pcm_fftMag_spectralRollOff90.0_sma', 'pcm_fftMag_spectralFlux_sma', 'pcm_fftMag_spectralCentroid_sma', 'pcm_fftMag_spectralEntropy_sma', 'pcm_fftMag_spectralVariance_sma', 'pcm_fftMag_spectralSkewness_sma', 'pcm_fftMag_spectralKurtosis_sma', 'pcm_fftMag_spectralSlope_sma', 'pcm_fftMag_psySharpness_sma', 'pcm_fftMag_spectralHarmonicity_sma', 'mfcc_sma[1]', 'mfcc_sma[3]', 'mfcc_sma[4]', 'mfcc_sma[5]', 'mfcc_sma[6]', 'mfcc_sma[7]', 'mfcc_sma[8]', 'mfcc_sma[9]', 'mfcc_sma[11]', 'mfcc_sma[12]', 'mfcc_sma[13]', 'mfcc_sma[14]', 'Loudness_sma3', 'alphaRatio_sma3', 'hammar

##Left vLPFC

In [ ]:
correlations_Left_vlPFC = powerAmplitudeAllLLDdf.corr()['Power: Left vlPFC'][4:]#.abs()


In [ ]:
positiveCorrelationCondition = correlations_Left_vlPFC > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Left_vlPFC < -1* rValFor95ConfidenceLevel
AcousticCorrelates95_Left_vlPFC = correlations_Left_vlPFC[ positiveCorrelationCondition | negativeCorrelationCondition]
AcousticCorrelates95_Left_vlPFC.head = 'r'

print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Left_vlPFC))
AcousticCorrelates95_Left_vlPFC.nlargest(len(AcousticCorrelates95_Left_vlPFC)) # for sorting

# of acoustic features where p < 0.05: 116


pcm_loudness_sma               0.180771
pcm_RMSenergy_sma              0.179618
spectralFlux_sma3              0.173008
pcm_fftMag_spectralFlux_sma    0.172264
audspec_lengthL1norm_sma       0.172151
                                 ...   
lspFreq_sma[1]                -0.174348
mfcc_sma[7]                   -0.187364
lspFreq_sma[2]                -0.189518
mfcc_sma[7].1                 -0.190234
F0env_sma                     -0.193372
Name: Power: Left vlPFC, Length: 116, dtype: float64

In [ ]:
AcousticCorrelates95_Left_vlPFC.nlargest(len(AcousticCorrelates95_Left_vlPFC))

In [ ]:
AcousticCorrelates95_Left_vlPFC.nlargest(len(AcousticCorrelates95_Left_vlPFC)).to_csv(date + '_noModeratorSpeech_AcousticCorrelates95_Left_vlPFC.csv',header=['r'],index_label='Acoustic Feature')

In [ ]:
print(AcousticCorrelates95_Left_vlPFC.index.tolist())

['F0final_sma', 'voicingFinalUnclipped_sma', 'jitterLocal_sma', 'jitterDDP_sma', 'shimmerLocal_sma', 'logHNR_sma', 'audspec_lengthL1norm_sma', 'audspecRasta_lengthL1norm_sma', 'pcm_RMSenergy_sma', 'pcm_zcr_sma', 'audSpec_Rfilt_sma[0]', 'audSpec_Rfilt_sma[1]', 'audSpec_Rfilt_sma[2]', 'audSpec_Rfilt_sma[3]', 'audSpec_Rfilt_sma[4]', 'audSpec_Rfilt_sma[5]', 'audSpec_Rfilt_sma[6]', 'audSpec_Rfilt_sma[7]', 'audSpec_Rfilt_sma[8]', 'audSpec_Rfilt_sma[9]', 'audSpec_Rfilt_sma[10]', 'audSpec_Rfilt_sma[11]', 'audSpec_Rfilt_sma[12]', 'audSpec_Rfilt_sma[13]', 'audSpec_Rfilt_sma[14]', 'audSpec_Rfilt_sma[15]', 'audSpec_Rfilt_sma[16]', 'audSpec_Rfilt_sma[17]', 'audSpec_Rfilt_sma[18]', 'audSpec_Rfilt_sma[19]', 'audSpec_Rfilt_sma[20]', 'audSpec_Rfilt_sma[21]', 'audSpec_Rfilt_sma[22]', 'audSpec_Rfilt_sma[23]', 'audSpec_Rfilt_sma[24]', 'audSpec_Rfilt_sma[25]', 'pcm_fftMag_fband250-650_sma', 'pcm_fftMag_fband1000-4000_sma', 'pcm_fftMag_spectralRollOff25.0_sma', 'pcm_fftMag_spectralRollOff50.0_sma', 'pcm_fft

####save best feature

In [ ]:
AcousticCorrelates95_Left_vlPFC.to_dict()['F0semitoneFrom27.5Hz_sma3nz']

0.1668217237012167

In [ ]:
powerAmplitudeAllLLDdf['F0semitoneFrom27.5Hz_sma3nz'].to_csv('Sept9LeftvLPFC_F0semitoneFrom27.5Hz_sma3nz.csv')

##Right OFC

In [ ]:
correlations_Right_OFC = powerAmplitudeAllLLDdf.corr()['Power: Right OFC'][4:]#.abs()


In [ ]:
positiveCorrelationCondition = correlations_Right_OFC > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Right_OFC < -1* rValFor95ConfidenceLevel
AcousticCorrelates95_Right_OFC = correlations_Right_OFC[ positiveCorrelationCondition | negativeCorrelationCondition]
AcousticCorrelates95_Right_OFC.head = 'r'

print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Right_OFC))
AcousticCorrelates95_Right_OFC.nlargest(len(AcousticCorrelates95_Right_OFC)) # for sorting

# of acoustic features where p < 0.05: 2


mfcc_sma[10].1   -0.072117
mfcc_sma[10]     -0.074690
Name: Power: Right OFC, dtype: float64

In [ ]:
AcousticCorrelates95_Right_OFC.nlargest(len(AcousticCorrelates95_Right_OFC)).to_csv(date + '_noModeratorSpeech_AcousticCorrelates95_Right_OFC.csv',header=['r'],index_label='Acoustic Feature')

In [ ]:
print(AcousticCorrelates95_Right_OFC.index.tolist())

['mfcc_sma[10]', 'mfcc_sma[10].1']


##Left OFC

In [ ]:
correlations_Left_OFC = powerAmplitudeAllLLDdf.corr()['Power: Left OFC'][8:]#.abs()


In [ ]:
positiveCorrelationCondition = correlations_Left_OFC > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Left_OFC < -1* rValFor95ConfidenceLevel
AcousticCorrelates95_Left_OFC = correlations_Left_OFC[ positiveCorrelationCondition | negativeCorrelationCondition]
AcousticCorrelates95_Left_OFC.head = 'r'

print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Left_OFC))
AcousticCorrelates95_Left_OFC.nlargest(len(AcousticCorrelates95_Left_OFC)) # for sorting

# of acoustic features where p < 0.05: 1


logRelF0-H1-H2_sma3nz    0.068429
Name: Power: Left OFC, dtype: float64

In [ ]:
AcousticCorrelates95_Left_OFC.nlargest(len(AcousticCorrelates95_Left_OFC)).to_csv(date + '_noModeratorSpeech_AcousticCorrelates95_Left_OFC.csv',header=['r'],index_label='Acoustic Feature')

In [ ]:
print(AcousticCorrelates95_Left_OFC.index.tolist())

['logRelF0-H1-H2_sma3nz']


#Sept 19

##Setup

In [ ]:
date = '2022-09-19'

In [ ]:
powerAmplitudeDF = pd.read_csv('2022-09-19-Power.csv')
del powerAmplitudeDF['Unnamed: 0']
powerAmplitudeDF.head()

,Power: Left OFC,Power: Left vlPFC,Power: Right OFC,Power: Right vlPFC
0,9.322238,4.995461,5.607341,3.754328
1,7.696407,7.638623,6.380779,7.017708
2,4.062073,4.581651,6.308548,9.426878
3,2.702953,3.492604,4.114555,7.404552
4,4.282622,5.795164,10.881743,3.557609


In [ ]:
powerAmplitudeDF.shape

(1137, 4)

In [ ]:
acousticFeatsDF = pd.read_csv('AcousticFeatures_noModeratorSpeech_aDBS012_' + '2022-09-19_' + 'audio_amplitude.csv')
del acousticFeatsDF['Unnamed: 0']
acousticFeatsDF.head()

,F0final_sma,voicingFinalUnclipped_sma,jitterLocal_sma,jitterDDP_sma,shimmerLocal_sma,logHNR_sma,audspec_lengthL1norm_sma,audspecRasta_lengthL1norm_sma,pcm_RMSenergy_sma,pcm_zcr_sma,...,lspFreq_sma_de[2],lspFreq_sma_de[3],lspFreq_sma_de[4],lspFreq_sma_de[5],lspFreq_sma_de[6],lspFreq_sma_de[7],pcm_zcr_sma_de.1,voiceProb_sma_de,F0_sma_de,F0env_sma_de
0,0.0,0.0,0.0,0.0,0.0,-100.0,0.001034,0.988341,0.0,0.999306,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,-100.0,0.001034,0.988341,0.0,0.999306,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,-100.0,0.001034,0.988341,0.0,0.999306,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,-100.0,0.001034,0.988341,0.0,0.999306,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,-100.0,0.001034,0.988341,0.0,0.999306,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
acousticFeatsDF.shape

(1129, 207)

In [ ]:
powerAmplitudeDF = powerAmplitudeDF.iloc[:acousticFeatsDF.shape[0]]
powerAmplitudeDF.shape

(1129, 4)

In [ ]:
powerAmplitudeAllLLDdf = pd.concat([powerAmplitudeDF,acousticFeatsDF],axis=1)
powerAmplitudeAllLLDdf.shape

(1129, 211)

Website to compute p value from r value and n: http://vassarstats.net/tabs_r.html

Need r >= 0.077 for p < 0.01
Need r >= 0.059 for p < 0.05

In [ ]:
rValFor99ConfidenceLevel = 0.077
rValFor95ConfidenceLevel = 0.059

##random noise t-test (10)

In [ ]:
import numpy as np

In [ ]:
# Set a seed for reproducibility (optional)
np.random.seed(37)

# Mean and standard deviation of the Gaussian distribution
mean_value = 0
std_dev = 1

# Generate a random element numpy array from a Gaussian distribution
random_array = np.random.normal(mean_value, std_dev, powerAmplitudeAllLLDdf.shape[0])
powerAmplitudeAllLLDdf['Noise'] = random_array
correlations_Noise = powerAmplitudeAllLLDdf.corr()['Noise'][4:-1]#.abs()

In [ ]:
positiveCorrelationCondition = correlations_Noise > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Noise < -1* rValFor95ConfidenceLevel
noiseCorrelates = correlations_Noise[ positiveCorrelationCondition | negativeCorrelationCondition]
noiseCorrelates.head = 'r'

print("# of acoustic features where p < 0.05:", len(noiseCorrelates))
noiseCorrelates.nlargest(len(noiseCorrelates)) # for sorting

# of acoustic features where p < 0.05: 10


lspFreq_sma[5]              0.069108
lspFreq_sma[4]              0.063187
F2bandwidth_sma3nz          0.061944
F3bandwidth_sma3nz          0.060692
lspFreq_sma[6]              0.060248
mfcc_sma[10]               -0.062081
mfcc_sma[8].1              -0.066255
audspec_lengthL1norm_sma   -0.066624
Loudness_sma3              -0.066624
mfcc_sma[8]                -0.070496
Name: Noise, dtype: float64

##Right vLPFC

In [ ]:
correlations_Right_vlPFC = powerAmplitudeAllLLDdf.corr()['Power: Right vlPFC'][4:]#.abs()

In [ ]:
positiveCorrelationCondition = correlations_Right_vlPFC > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Right_vlPFC < -1* rValFor95ConfidenceLevel
AcousticCorrelates95_Right_vlPFC = correlations_Right_vlPFC[ positiveCorrelationCondition | negativeCorrelationCondition]
AcousticCorrelates95_Right_vlPFC.head = 'r'

print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Right_vlPFC))
AcousticCorrelates95_Right_vlPFC.nlargest(len(AcousticCorrelates95_Right_vlPFC)) # for sorting

# of acoustic features where p < 0.05: 51


mfcc1_sma3                        0.084081
mfcc_sma[1]                       0.084081
Loudness_sma3                     0.082162
audspec_lengthL1norm_sma          0.082162
pcm_fftMag_spectralEntropy_sma    0.082039
F2bandwidth_sma3nz                0.082010
pcm_RMSenergy_sma                 0.081185
F1bandwidth_sma3nz                0.079644
F2amplitudeLogRelF0_sma3nz        0.079138
F3amplitudeLogRelF0_sma3nz        0.078433
F1amplitudeLogRelF0_sma3nz        0.078423
pcm_loudness_sma                  0.077944
spectralFlux_sma3                 0.077683
pcm_fftMag_spectralFlux_sma       0.076818
logHNR_sma                        0.076766
voicingFinalUnclipped_sma         0.075076
F0semitoneFrom27.5Hz_sma3nz       0.074107
F3bandwidth_sma3nz                0.073033
pcm_fftMag_fband250-650_sma       0.072930
F3frequency_sma3nz                0.071313
mfcc_sma[1].1                     0.070681
shimmerLocal_sma                  0.070630
hammarbergIndex_sma3              0.070151
F0final_sma

In [ ]:
AcousticCorrelates95_Right_vlPFC.nlargest(len(AcousticCorrelates95_Right_vlPFC)).to_csv(date + '_noModeratorSpeech_AcousticCorrelates95_Right_vlPFC.csv',header=['r'],index_label='Acoustic Feature')

In [ ]:
print(AcousticCorrelates95_Right_vlPFC.index.tolist())

['F0final_sma', 'voicingFinalUnclipped_sma', 'jitterLocal_sma', 'shimmerLocal_sma', 'logHNR_sma', 'audspec_lengthL1norm_sma', 'pcm_RMSenergy_sma', 'pcm_zcr_sma', 'pcm_fftMag_fband250-650_sma', 'pcm_fftMag_spectralFlux_sma', 'pcm_fftMag_spectralEntropy_sma', 'pcm_fftMag_spectralSlope_sma', 'pcm_fftMag_psySharpness_sma', 'mfcc_sma[1]', 'mfcc_sma[11]', 'mfcc_sma[12]', 'Loudness_sma3', 'alphaRatio_sma3', 'hammarbergIndex_sma3', 'slope0-500_sma3', 'slope500-1500_sma3', 'spectralFlux_sma3', 'mfcc1_sma3', 'F0semitoneFrom27.5Hz_sma3nz', 'shimmerLocaldB_sma3nz', 'logRelF0-H1-A3_sma3nz', 'F1frequency_sma3nz', 'F1bandwidth_sma3nz', 'F1amplitudeLogRelF0_sma3nz', 'F2frequency_sma3nz', 'F2bandwidth_sma3nz', 'F2amplitudeLogRelF0_sma3nz', 'F3frequency_sma3nz', 'F3bandwidth_sma3nz', 'F3amplitudeLogRelF0_sma3nz', 'pcm_intensity_sma', 'pcm_loudness_sma', 'mfcc_sma[1].1', 'mfcc_sma[2].1', 'mfcc_sma[12].1', 'lspFreq_sma[0]', 'lspFreq_sma[1]', 'lspFreq_sma[2]', 'lspFreq_sma[3]', 'lspFreq_sma[4]', 'lspFreq_s

##Left vLPFC

In [ ]:
correlations_Left_vlPFC = powerAmplitudeAllLLDdf.corr()['Power: Left vlPFC'][4:]#.abs()


In [ ]:
positiveCorrelationCondition = correlations_Left_vlPFC > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Left_vlPFC < -1* rValFor95ConfidenceLevel
AcousticCorrelates95_Left_vlPFC = correlations_Left_vlPFC[ positiveCorrelationCondition | negativeCorrelationCondition]
AcousticCorrelates95_Left_vlPFC.head = 'r'

print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Left_vlPFC))
AcousticCorrelates95_Left_vlPFC.nlargest(len(AcousticCorrelates95_Left_vlPFC)) # for sorting

# of acoustic features where p < 0.05: 97


logHNR_sma_de          0.156419
shimmerLocal_sma_de    0.155074
jitterLocal_sma_de     0.147936
F0env_sma              0.127801
jitterDDP_sma_de       0.117138
                         ...   
lspFreq_sma[7]        -0.102809
lspFreq_sma[5]        -0.103208
slope0-500_sma3       -0.103946
mfcc_sma[12].1        -0.104564
F0final_sma_de        -0.121611
Name: Power: Left vlPFC, Length: 97, dtype: float64

In [ ]:
AcousticCorrelates95_Left_vlPFC.nlargest(len(AcousticCorrelates95_Left_vlPFC)).to_csv(date + '_noModeratorSpeech_AcousticCorrelates95_Left_vlPFC.csv',header=['r'],index_label='Acoustic Feature')

In [ ]:
print(AcousticCorrelates95_Left_vlPFC.index.tolist())

['F0final_sma', 'voicingFinalUnclipped_sma', 'jitterLocal_sma', 'jitterDDP_sma', 'shimmerLocal_sma', 'logHNR_sma', 'audspec_lengthL1norm_sma', 'audspecRasta_lengthL1norm_sma', 'pcm_RMSenergy_sma', 'pcm_zcr_sma', 'audSpec_Rfilt_sma[0]', 'audSpec_Rfilt_sma[2]', 'audSpec_Rfilt_sma[5]', 'audSpec_Rfilt_sma[6]', 'audSpec_Rfilt_sma[7]', 'audSpec_Rfilt_sma[8]', 'audSpec_Rfilt_sma[9]', 'audSpec_Rfilt_sma[10]', 'audSpec_Rfilt_sma[11]', 'audSpec_Rfilt_sma[12]', 'audSpec_Rfilt_sma[13]', 'audSpec_Rfilt_sma[14]', 'audSpec_Rfilt_sma[15]', 'audSpec_Rfilt_sma[17]', 'audSpec_Rfilt_sma[18]', 'audSpec_Rfilt_sma[19]', 'audSpec_Rfilt_sma[20]', 'audSpec_Rfilt_sma[21]', 'audSpec_Rfilt_sma[22]', 'audSpec_Rfilt_sma[23]', 'audSpec_Rfilt_sma[24]', 'audSpec_Rfilt_sma[25]', 'pcm_fftMag_fband1000-4000_sma', 'pcm_fftMag_spectralRollOff25.0_sma', 'pcm_fftMag_spectralRollOff50.0_sma', 'pcm_fftMag_spectralRollOff75.0_sma', 'pcm_fftMag_spectralRollOff90.0_sma', 'pcm_fftMag_spectralFlux_sma', 'pcm_fftMag_spectralCentroid_

##Right OFC

In [ ]:
correlations_Right_OFC = powerAmplitudeAllLLDdf.corr()['Power: Right OFC'][4:]#.abs()


In [ ]:
positiveCorrelationCondition = correlations_Right_OFC > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Right_OFC < -1* rValFor95ConfidenceLevel
AcousticCorrelates95_Right_OFC = correlations_Right_OFC[ positiveCorrelationCondition | negativeCorrelationCondition]
AcousticCorrelates95_Right_OFC.head = 'r'

print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Right_OFC))
AcousticCorrelates95_Right_OFC.nlargest(len(AcousticCorrelates95_Right_OFC)) # for sorting

# of acoustic features where p < 0.05: 3


voicingFinalUnclipped_sma_de   -0.059220
audSpec_Rfilt_sma[13]          -0.060517
audSpec_Rfilt_sma[25]          -0.060821
Name: Power: Right OFC, dtype: float64

In [ ]:
AcousticCorrelates95_Right_OFC.nlargest(len(AcousticCorrelates95_Right_OFC)).to_csv(date + '_noModeratorSpeech_AcousticCorrelates95_Right_OFC.csv',header=['r'],index_label='Acoustic Feature')

In [ ]:
print(AcousticCorrelates95_Right_OFC.index.tolist())

['audSpec_Rfilt_sma[13]', 'audSpec_Rfilt_sma[25]', 'voicingFinalUnclipped_sma_de']


##Left OFC

In [ ]:
correlations_Left_OFC = powerAmplitudeAllLLDdf.corr()['Power: Left OFC'][8:]#.abs()


In [ ]:
positiveCorrelationCondition = correlations_Left_OFC > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Left_OFC < -1* rValFor95ConfidenceLevel
AcousticCorrelates95_Left_OFC = correlations_Left_OFC[ positiveCorrelationCondition | negativeCorrelationCondition]
AcousticCorrelates95_Left_OFC.head = 'r'

print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Left_OFC))
AcousticCorrelates95_Left_OFC.nlargest(len(AcousticCorrelates95_Left_OFC)) # for sorting

# of acoustic features where p < 0.05: 8


mfcc_sma[13]                          0.088447
pcm_fftMag_fband250-650_sma           0.067530
pcm_fftMag_spectralHarmonicity_sma    0.063786
pcm_fftMag_fband1000-4000_sma         0.062223
pcm_intensity_sma                     0.060588
pcm_fftMag_spectralSlope_sma         -0.064465
mfcc_sma[7]                          -0.081689
mfcc_sma[7].1                        -0.082362
Name: Power: Left OFC, dtype: float64

In [ ]:
AcousticCorrelates95_Left_OFC.nlargest(len(AcousticCorrelates95_Left_OFC)).to_csv(date + '_noModeratorSpeech_AcousticCorrelates95_Left_OFC.csv',header=['r'],index_label='Acoustic Feature')

In [ ]:
print(AcousticCorrelates95_Left_OFC.index.tolist())

['pcm_fftMag_fband250-650_sma', 'pcm_fftMag_fband1000-4000_sma', 'pcm_fftMag_spectralSlope_sma', 'pcm_fftMag_spectralHarmonicity_sma', 'mfcc_sma[7]', 'mfcc_sma[13]', 'pcm_intensity_sma', 'mfcc_sma[7].1']


#Oct 4

##Setup

In [ ]:
date = '2022-10-04'

In [ ]:
powerAmplitudeDF = pd.read_csv('2022-10-04-Power-Amplitude.csv')
del powerAmplitudeDF['Unnamed: 0']
powerAmplitudeDF.head()

,Power: Left OFC,Power: Left vlPFC,Power: Right OFC,Power: Right vlPFC
0,6.407832,8.696524,10.436554,11.400055
1,6.523042,8.672893,8.723282,10.050933
2,6.641233,10.282269,8.287294,7.250344
3,3.958534,8.233730,10.518788,10.198656
4,8.917095,3.029191,9.664642,4.365085


In [ ]:
powerAmplitudeDF.shape

(1154, 4)

In [ ]:
acousticFeatsDF = pd.read_csv('AcousticFeatures_noModeratorSpeech_aDBS012_' + '2022_10_04_' + 'audio_amplitude.csv')
del acousticFeatsDF['Unnamed: 0']
acousticFeatsDF.head()

,F0final_sma,voicingFinalUnclipped_sma,jitterLocal_sma,jitterDDP_sma,shimmerLocal_sma,logHNR_sma,audspec_lengthL1norm_sma,audspecRasta_lengthL1norm_sma,pcm_RMSenergy_sma,pcm_zcr_sma,...,lspFreq_sma_de[2],lspFreq_sma_de[3],lspFreq_sma_de[4],lspFreq_sma_de[5],lspFreq_sma_de[6],lspFreq_sma_de[7],pcm_zcr_sma_de.1,voiceProb_sma_de,F0_sma_de,F0env_sma_de
0,0.0,0.0,0.0,0.0,0.0,-100.0,0.001034,0.988341,0.0,0.999244,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,-100.0,0.001034,0.988341,0.0,0.999244,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,-100.0,0.001034,0.988341,0.0,0.999244,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,-100.0,0.001034,0.988341,0.0,0.999244,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,-100.0,0.001034,0.988341,0.0,0.999244,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
acousticFeatsDF.shape

(1107, 207)

In [ ]:
powerAmplitudeDF = powerAmplitudeDF.iloc[:acousticFeatsDF.shape[0]]
powerAmplitudeDF.shape

(1107, 4)

In [ ]:
powerAmplitudeAllLLDdf = pd.concat([powerAmplitudeDF,acousticFeatsDF],axis=1)
powerAmplitudeAllLLDdf.shape

(1107, 211)

Website to compute p value from r value and n: http://vassarstats.net/tabs_r.html

In [ ]:
rValFor99ConfidenceLevel = 0.078
rValFor95ConfidenceLevel = 0.059

##random noise t-test (10)

In [ ]:
import numpy as np

In [ ]:
# Set a seed for reproducibility
np.random.seed(37)

# Mean and standard deviation of the Gaussian distribution
mean_value = 0
std_dev = 1

# Generate a random element numpy array from a Gaussian distribution
random_array = np.random.normal(mean_value, std_dev, powerAmplitudeAllLLDdf.shape[0])
powerAmplitudeAllLLDdf['Noise'] = random_array
correlations_Noise = powerAmplitudeAllLLDdf.corr()['Noise'][4:-1]#.abs()

In [ ]:
positiveCorrelationCondition = correlations_Noise > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Noise < -1* rValFor95ConfidenceLevel
noiseCorrelates = correlations_Noise[ positiveCorrelationCondition | negativeCorrelationCondition]
noiseCorrelates.head = 'r'

print("# of acoustic features where p < 0.05:", len(noiseCorrelates))
noiseCorrelates.nlargest(len(noiseCorrelates)) # for sorting

# of acoustic features where p < 0.05: 10


lspFreq_sma[5]              0.069108
lspFreq_sma[4]              0.063187
F2bandwidth_sma3nz          0.061944
F3bandwidth_sma3nz          0.060692
lspFreq_sma[6]              0.060248
mfcc_sma[10]               -0.062081
mfcc_sma[8].1              -0.066255
audspec_lengthL1norm_sma   -0.066624
Loudness_sma3              -0.066624
mfcc_sma[8]                -0.070496
Name: Noise, dtype: float64

##Right vLPFC

In [ ]:
correlations_Right_vlPFC = powerAmplitudeAllLLDdf.corr()['Power: Right vlPFC'][4:]#.abs()

In [ ]:
positiveCorrelationCondition = correlations_Right_vlPFC > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Right_vlPFC < -1* rValFor95ConfidenceLevel
AcousticCorrelates95_Right_vlPFC = correlations_Right_vlPFC[ positiveCorrelationCondition | negativeCorrelationCondition]
AcousticCorrelates95_Right_vlPFC.head = 'r'

print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Right_vlPFC))
AcousticCorrelates95_Right_vlPFC.nlargest(len(AcousticCorrelates95_Right_vlPFC)) # for sorting

# of acoustic features where p < 0.05: 40


pcm_fftMag_spectralRollOff50.0_sma    0.088087
mfcc_sma[8].1                         0.087795
pcm_fftMag_spectralRollOff75.0_sma    0.087308
mfcc_sma[8]                           0.085637
pcm_fftMag_spectralRollOff90.0_sma    0.084122
Loudness_sma3                         0.083883
audspec_lengthL1norm_sma              0.083883
pcm_fftMag_spectralFlux_sma           0.081387
pcm_fftMag_psySharpness_sma           0.080063
pcm_fftMag_spectralCentroid_sma       0.079757
spectralFlux_sma3                     0.079049
pcm_loudness_sma                      0.078300
pcm_RMSenergy_sma                     0.078286
pcm_fftMag_spectralRollOff25.0_sma    0.070786
pcm_fftMag_fband250-650_sma           0.070606
HNRdBACF_sma3nz                       0.070094
logRelF0-H1-A3_sma3nz                 0.067422
pcm_fftMag_spectralEntropy_sma        0.067315
F1frequency_sma3nz                    0.064594
F0_sma                                0.061124
voiceProb_sma                         0.060407
F0semitoneFro

In [ ]:
AcousticCorrelates95_Right_vlPFC.nlargest(len(AcousticCorrelates95_Right_vlPFC)).to_csv(date + '_noModeratorSpeech_AcousticCorrelates95_Right_vlPFC.csv',header=['r'],index_label='Acoustic Feature')

In [ ]:
print(AcousticCorrelates95_Right_vlPFC.index.tolist())

['voicingFinalUnclipped_sma', 'audspec_lengthL1norm_sma', 'pcm_RMSenergy_sma', 'pcm_fftMag_fband250-650_sma', 'pcm_fftMag_spectralRollOff25.0_sma', 'pcm_fftMag_spectralRollOff50.0_sma', 'pcm_fftMag_spectralRollOff75.0_sma', 'pcm_fftMag_spectralRollOff90.0_sma', 'pcm_fftMag_spectralFlux_sma', 'pcm_fftMag_spectralCentroid_sma', 'pcm_fftMag_spectralEntropy_sma', 'pcm_fftMag_psySharpness_sma', 'mfcc_sma[6]', 'mfcc_sma[7]', 'mfcc_sma[8]', 'mfcc_sma[9]', 'Loudness_sma3', 'slope500-1500_sma3', 'spectralFlux_sma3', 'F0semitoneFrom27.5Hz_sma3nz', 'HNRdBACF_sma3nz', 'logRelF0-H1-A3_sma3nz', 'F1frequency_sma3nz', 'F2frequency_sma3nz', 'F3frequency_sma3nz', 'pcm_loudness_sma', 'mfcc_sma[5].1', 'mfcc_sma[6].1', 'mfcc_sma[7].1', 'mfcc_sma[8].1', 'mfcc_sma[9].1', 'lspFreq_sma[1]', 'lspFreq_sma[2]', 'lspFreq_sma[3]', 'lspFreq_sma[4]', 'lspFreq_sma[5]', 'lspFreq_sma[6]', 'voiceProb_sma', 'F0_sma', 'F0env_sma']


##Left vLPFC

In [ ]:
correlations_Left_vlPFC = powerAmplitudeAllLLDdf.corr()['Power: Left vlPFC'][4:]#.abs()
correlations_Left_vlPFC

F0final_sma                  0.088180
voicingFinalUnclipped_sma    0.083800
jitterLocal_sma              0.045865
jitterDDP_sma                0.050136
shimmerLocal_sma             0.085279
                               ...   
lspFreq_sma_de[7]           -0.011069
pcm_zcr_sma_de.1            -0.011147
voiceProb_sma_de             0.018981
F0_sma_de                    0.031685
F0env_sma_de                 0.003743
Name: Power: Left vlPFC, Length: 207, dtype: float64

In [ ]:
positiveCorrelationCondition = correlations_Left_vlPFC > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Left_vlPFC < -1* rValFor95ConfidenceLevel
AcousticCorrelates95_Left_vlPFC = correlations_Left_vlPFC[ positiveCorrelationCondition | negativeCorrelationCondition]
AcousticCorrelates95_Left_vlPFC.head = 'r'

print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Left_vlPFC))
AcousticCorrelates95_Left_vlPFC.nlargest(len(AcousticCorrelates95_Left_vlPFC)) # for sorting

# of acoustic features where p < 0.05: 72


shimmerLocaldB_sma3nz              0.104745
pcm_fftMag_spectralKurtosis_sma    0.103678
pcm_fftMag_spectralSkewness_sma    0.100690
mfcc1_sma3                         0.093991
mfcc_sma[1]                        0.093991
                                     ...   
mfcc_sma[5].1                     -0.091260
mfcc_sma[6].1                     -0.092823
mfcc_sma[6]                       -0.096620
lspFreq_sma[1]                    -0.097931
F0env_sma                         -0.118623
Name: Power: Left vlPFC, Length: 72, dtype: float64

In [ ]:
AcousticCorrelates95_Left_vlPFC.nlargest(len(AcousticCorrelates95_Left_vlPFC)).to_csv(date + '_noModeratorSpeech_AcousticCorrelates95_Left_vlPFC.csv',header=['r'],index_label='Acoustic Feature')

In [ ]:
print(AcousticCorrelates95_Left_vlPFC.index.tolist())

['F0final_sma', 'voicingFinalUnclipped_sma', 'shimmerLocal_sma', 'logHNR_sma', 'audspec_lengthL1norm_sma', 'pcm_RMSenergy_sma', 'pcm_zcr_sma', 'audSpec_Rfilt_sma[0]', 'audSpec_Rfilt_sma[1]', 'audSpec_Rfilt_sma[2]', 'audSpec_Rfilt_sma[3]', 'audSpec_Rfilt_sma[4]', 'audSpec_Rfilt_sma[12]', 'audSpec_Rfilt_sma[13]', 'audSpec_Rfilt_sma[14]', 'audSpec_Rfilt_sma[15]', 'pcm_fftMag_spectralRollOff75.0_sma', 'pcm_fftMag_spectralRollOff90.0_sma', 'pcm_fftMag_spectralFlux_sma', 'pcm_fftMag_spectralEntropy_sma', 'pcm_fftMag_spectralSkewness_sma', 'pcm_fftMag_spectralKurtosis_sma', 'pcm_fftMag_psySharpness_sma', 'mfcc_sma[1]', 'mfcc_sma[3]', 'mfcc_sma[4]', 'mfcc_sma[5]', 'mfcc_sma[6]', 'mfcc_sma[7]', 'mfcc_sma[9]', 'Loudness_sma3', 'alphaRatio_sma3', 'hammarbergIndex_sma3', 'slope0-500_sma3', 'slope500-1500_sma3', 'spectralFlux_sma3', 'mfcc1_sma3', 'mfcc3_sma3', 'mfcc4_sma3', 'F0semitoneFrom27.5Hz_sma3nz', 'jitterLocal_sma3nz', 'shimmerLocaldB_sma3nz', 'logRelF0-H1-A3_sma3nz', 'F1frequency_sma3nz', '

##Right OFC

In [ ]:
correlations_Right_OFC = powerAmplitudeAllLLDdf.corr()['Power: Right OFC'][4:]#.abs()


In [ ]:
positiveCorrelationCondition = correlations_Right_OFC > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Right_OFC < -1* rValFor95ConfidenceLevel
AcousticCorrelates95_Right_OFC = correlations_Right_OFC[ positiveCorrelationCondition | negativeCorrelationCondition]
AcousticCorrelates95_Right_OFC.head = 'r'

print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Right_OFC))
AcousticCorrelates95_Right_OFC.nlargest(len(AcousticCorrelates95_Right_OFC)) # for sorting

# of acoustic features where p < 0.05: 21


lspFreq_sma[0]                      0.074761
audSpec_Rfilt_sma_de[4]             0.071167
audSpec_Rfilt_sma_de[8]             0.069687
audSpec_Rfilt_sma_de[5]             0.067768
audSpec_Rfilt_sma_de[9]             0.067382
audspecRasta_lengthL1norm_sma_de    0.065424
audSpec_Rfilt_sma_de[17]            0.065191
F0final_sma_de                      0.065085
audSpec_Rfilt_sma_de[12]            0.064882
audSpec_Rfilt_sma_de[15]            0.063643
audSpec_Rfilt_sma_de[18]            0.063100
audSpec_Rfilt_sma_de[11]            0.063094
audSpec_Rfilt_sma_de[7]             0.062755
audSpec_Rfilt_sma_de[16]            0.062647
audSpec_Rfilt_sma_de[13]            0.062092
audSpec_Rfilt_sma_de[6]             0.061125
audSpec_Rfilt_sma_de[14]            0.061067
audSpec_Rfilt_sma_de[10]            0.061029
audSpec_Rfilt_sma_de[3]             0.060688
audSpec_Rfilt_sma_de[19]            0.059240
mfcc_sma[1].1                      -0.063047
Name: Power: Right OFC, dtype: float64

In [ ]:
AcousticCorrelates95_Right_OFC.nlargest(len(AcousticCorrelates95_Right_OFC)).to_csv(date + '_noModeratorSpeech_AcousticCorrelates95_Right_OFC.csv',header=['r'],index_label='Acoustic Feature')

In [ ]:
print(AcousticCorrelates95_Right_OFC.index.tolist())

['mfcc_sma[1].1', 'lspFreq_sma[0]', 'F0final_sma_de', 'audspecRasta_lengthL1norm_sma_de', 'audSpec_Rfilt_sma_de[3]', 'audSpec_Rfilt_sma_de[4]', 'audSpec_Rfilt_sma_de[5]', 'audSpec_Rfilt_sma_de[6]', 'audSpec_Rfilt_sma_de[7]', 'audSpec_Rfilt_sma_de[8]', 'audSpec_Rfilt_sma_de[9]', 'audSpec_Rfilt_sma_de[10]', 'audSpec_Rfilt_sma_de[11]', 'audSpec_Rfilt_sma_de[12]', 'audSpec_Rfilt_sma_de[13]', 'audSpec_Rfilt_sma_de[14]', 'audSpec_Rfilt_sma_de[15]', 'audSpec_Rfilt_sma_de[16]', 'audSpec_Rfilt_sma_de[17]', 'audSpec_Rfilt_sma_de[18]', 'audSpec_Rfilt_sma_de[19]']


##Left OFC

In [ ]:
correlations_Left_OFC = powerAmplitudeAllLLDdf.corr()['Power: Left OFC'][8:]#.abs()


In [ ]:
positiveCorrelationCondition = correlations_Left_OFC > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Left_OFC < -1* rValFor95ConfidenceLevel
AcousticCorrelates95_Left_OFC = correlations_Left_OFC[ positiveCorrelationCondition | negativeCorrelationCondition]
AcousticCorrelates95_Left_OFC.head = 'r'

print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Left_OFC))
AcousticCorrelates95_Left_OFC.nlargest(len(AcousticCorrelates95_Left_OFC)) # for sorting

# of acoustic features where p < 0.05: 1


pcm_fftMag_spectralEntropy_sma_de    0.06255
Name: Power: Left OFC, dtype: float64

In [ ]:
AcousticCorrelates95_Left_OFC.nlargest(len(AcousticCorrelates95_Left_OFC)).to_csv(date + '_noModeratorSpeech_AcousticCorrelates95_Left_OFC.csv',header=['r'],index_label='Acoustic Feature')

In [ ]:
print(AcousticCorrelates95_Left_OFC.index.tolist())

['pcm_fftMag_spectralEntropy_sma_de']


#Nov 15

##Setup

In [ ]:
date ='2022-11-15'

In [ ]:
powerDF = pd.read_csv('2022-11-15-Power.csv')
del powerDF['Unnamed: 0']
powerDF.head()

,Power: Left OFC,Power: Left vlPFC,Power: Right OFC,Power: Right vlPFC
0,8.462996,6.523769,4.619912,8.098723
1,5.854915,5.621985,7.196563,7.780724
2,9.335134,7.215692,7.966418,9.071778
3,4.709172,5.349554,7.623001,11.709882
4,5.242306,13.146638,7.908021,7.542927


In [ ]:
powerDF.shape

(1143, 4)

In [ ]:
acousticFeatsDF = pd.read_csv('AcousticFeatures_noModeratorSpeech_aDBS012_' + '2022-11-15_' + 'audio_amplitude.csv')
del acousticFeatsDF['Unnamed: 0']
acousticFeatsDF.head()

,F0final_sma,voicingFinalUnclipped_sma,jitterLocal_sma,jitterDDP_sma,shimmerLocal_sma,logHNR_sma,audspec_lengthL1norm_sma,audspecRasta_lengthL1norm_sma,pcm_RMSenergy_sma,pcm_zcr_sma,...,lspFreq_sma_de[2],lspFreq_sma_de[3],lspFreq_sma_de[4],lspFreq_sma_de[5],lspFreq_sma_de[6],lspFreq_sma_de[7],pcm_zcr_sma_de.1,voiceProb_sma_de,F0_sma_de,F0env_sma_de
0,0.0,0.0,0.0,0.0,0.0,-100.0,0.001034,0.988341,0.0,0.999244,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,-100.0,0.001034,0.988341,0.0,0.999244,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,-100.0,0.001034,0.988341,0.0,0.999244,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,-100.0,0.001034,0.988341,0.0,0.999244,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,-100.0,0.001034,0.988341,0.0,0.999244,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
acousticFeatsDF.shape

(1101, 207)

In [ ]:
powerDF = powerDF.iloc[:acousticFeatsDF.shape[0]]
powerDF.shape

(1101, 4)

In [ ]:
powerAllLLDdf = pd.concat([powerDF,acousticFeatsDF],axis=1)
powerAllLLDdf.shape

(1101, 211)

Website to compute p value from r value and n: http://vassarstats.net/tabs_r.html

Need r >= 0.077 for p < 0.01
Need r >= 0.059 for p < 0.05

In [ ]:
rValFor99ConfidenceLevel = 0.078
rValFor95ConfidenceLevel = 0.0591

##random noise t-test (10)

In [ ]:
import numpy as np

In [ ]:
# Set a seed for reproducibility (optional)
np.random.seed(33)

# Mean and standard deviation of the Gaussian distribution
mean_value = 0
std_dev = 1

# Generate a random element numpy array from a Gaussian distribution
random_array = np.random.normal(mean_value, std_dev, powerAllLLDdf.shape[0])


In [ ]:
powerAllLLDdf['Noise'] = random_array

In [ ]:
powerAllLLDdf.corr()['Noise'][4:-1]#.abs()

F0final_sma                  0.027509
voicingFinalUnclipped_sma    0.012343
jitterLocal_sma             -0.012747
jitterDDP_sma               -0.007248
shimmerLocal_sma             0.002880
                               ...   
lspFreq_sma_de[7]           -0.016493
pcm_zcr_sma_de.1            -0.013941
voiceProb_sma_de             0.024742
F0_sma_de                    0.046550
F0env_sma_de                 0.019065
Name: Noise, Length: 207, dtype: float64

In [ ]:
correlations_Noise = powerAllLLDdf.corr()['Noise'][4:-1]#.abs()

In [ ]:
positiveCorrelationCondition = correlations_Noise > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Noise < -1* rValFor95ConfidenceLevel
noiseCorrelates = correlations_Noise[ positiveCorrelationCondition | negativeCorrelationCondition]
noiseCorrelates.head = 'r'

print("# of acoustic features where p < 0.05:", len(noiseCorrelates))
noiseCorrelates.nlargest(len(noiseCorrelates)) # for sorting

# of acoustic features where p < 0.05: 10


audSpec_Rfilt_sma[16]              0.078404
audSpec_Rfilt_sma[15]              0.076071
audSpec_Rfilt_sma[17]              0.072213
audSpec_Rfilt_sma[14]              0.071498
audSpec_Rfilt_sma[13]              0.067982
pcm_fftMag_spectralKurtosis_sma    0.062918
pcm_fftMag_fband1000-4000_sma      0.062300
F1frequency_sma3nz                 0.062186
audspecRasta_lengthL1norm_sma      0.061742
audSpec_Rfilt_sma[6]               0.061595
Name: Noise, dtype: float64

##Right vLPFC

In [ ]:
correlations_Right_vlPFC = powerAllLLDdf.corr()['Power: Right vlPFC'][4:]#.abs()

In [ ]:
positiveCorrelationCondition = correlations_Right_vlPFC > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Right_vlPFC < -1* rValFor95ConfidenceLevel
AcousticCorrelates95_Right_vlPFC = correlations_Right_vlPFC[ positiveCorrelationCondition | negativeCorrelationCondition]
AcousticCorrelates95_Right_vlPFC.head = 'r'

print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Right_vlPFC))
AcousticCorrelates95_Right_vlPFC.nlargest(len(AcousticCorrelates95_Right_vlPFC)) # for sorting

# of acoustic features where p < 0.05: 40


jitterDDP_sma                      0.085909
shimmerLocal_sma                   0.079055
jitterLocal_sma3nz                 0.078684
shimmerLocaldB_sma3nz              0.078375
jitterLocal_sma                    0.076038
F0_sma                             0.074130
F2bandwidth_sma3nz                 0.074070
pcm_fftMag_spectralSkewness_sma    0.072779
F2amplitudeLogRelF0_sma3nz         0.071586
logHNR_sma                         0.071481
F1amplitudeLogRelF0_sma3nz         0.070749
F3amplitudeLogRelF0_sma3nz         0.070367
hammarbergIndex_sma3               0.068822
mfcc_sma[1]                        0.067843
mfcc1_sma3                         0.067843
F0semitoneFrom27.5Hz_sma3nz        0.067286
F0final_sma                        0.067221
audSpec_Rfilt_sma[10]              0.067129
F3bandwidth_sma3nz                 0.066710
F1frequency_sma3nz                 0.066417
pcm_fftMag_spectralKurtosis_sma    0.065957
F1bandwidth_sma3nz                 0.065787
voicingFinalUnclipped_sma       

In [ ]:
AcousticCorrelates95_Right_vlPFC.nlargest(len(AcousticCorrelates95_Right_vlPFC)).to_csv(date + '_noModeratorSpeech_AcousticCorrelates95_Right_vlPFC.csv',header=['r'],index_label='Acoustic Feature')

In [ ]:
print(AcousticCorrelates95_Right_vlPFC.index.tolist())

['F0final_sma', 'voicingFinalUnclipped_sma', 'jitterLocal_sma', 'jitterDDP_sma', 'shimmerLocal_sma', 'logHNR_sma', 'pcm_zcr_sma', 'audSpec_Rfilt_sma[9]', 'audSpec_Rfilt_sma[10]', 'audSpec_Rfilt_sma[14]', 'audSpec_Rfilt_sma[17]', 'audSpec_Rfilt_sma[18]', 'pcm_fftMag_spectralSkewness_sma', 'pcm_fftMag_spectralKurtosis_sma', 'mfcc_sma[1]', 'mfcc_sma[3]', 'mfcc_sma[7]', 'alphaRatio_sma3', 'hammarbergIndex_sma3', 'slope0-500_sma3', 'mfcc1_sma3', 'mfcc3_sma3', 'F0semitoneFrom27.5Hz_sma3nz', 'jitterLocal_sma3nz', 'shimmerLocaldB_sma3nz', 'F1frequency_sma3nz', 'F1bandwidth_sma3nz', 'F1amplitudeLogRelF0_sma3nz', 'F2frequency_sma3nz', 'F2bandwidth_sma3nz', 'F2amplitudeLogRelF0_sma3nz', 'F3frequency_sma3nz', 'F3bandwidth_sma3nz', 'F3amplitudeLogRelF0_sma3nz', 'mfcc_sma[7].1', 'lspFreq_sma[0]', 'lspFreq_sma[1]', 'pcm_zcr_sma.1', 'voiceProb_sma', 'F0_sma']


##Left vLPFC

In [ ]:
correlations_Left_vlPFC = powerAllLLDdf.corr()['Power: Left vlPFC'][4:]#.abs()
correlations_Left_vlPFC

F0final_sma                  0.110506
voicingFinalUnclipped_sma    0.113462
jitterLocal_sma              0.028394
jitterDDP_sma                0.031676
shimmerLocal_sma             0.082948
                               ...   
lspFreq_sma_de[7]            0.002062
pcm_zcr_sma_de.1             0.008610
voiceProb_sma_de            -0.009658
F0_sma_de                   -0.025948
F0env_sma_de                -0.038460
Name: Power: Left vlPFC, Length: 207, dtype: float64

In [ ]:
positiveCorrelationCondition = correlations_Left_vlPFC > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Left_vlPFC < -1* rValFor95ConfidenceLevel
AcousticCorrelates95_Left_vlPFC = correlations_Left_vlPFC[ positiveCorrelationCondition | negativeCorrelationCondition]
AcousticCorrelates95_Left_vlPFC.head = 'r'

print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Left_vlPFC))
AcousticCorrelates95_Left_vlPFC.nlargest(len(AcousticCorrelates95_Left_vlPFC)) # for sorting

# of acoustic features where p < 0.05: 88


pcm_fftMag_spectralRollOff50.0_sma    0.153284
pcm_fftMag_psySharpness_sma           0.151970
pcm_fftMag_spectralRollOff90.0_sma    0.151965
pcm_fftMag_spectralCentroid_sma       0.150623
pcm_fftMag_spectralRollOff75.0_sma    0.150435
                                        ...   
lspFreq_sma[6]                       -0.146092
lspFreq_sma[4]                       -0.147611
lspFreq_sma[7]                       -0.151307
F0env_sma                            -0.155317
lspFreq_sma[5]                       -0.155372
Name: Power: Left vlPFC, Length: 88, dtype: float64

In [ ]:
AcousticCorrelates95_Left_vlPFC.nlargest(len(AcousticCorrelates95_Left_vlPFC)).to_csv(date + '_noModeratorSpeech_AcousticCorrelates95_Left_vlPFC.csv',header=['r'],index_label='Acoustic Feature')

In [ ]:
print(AcousticCorrelates95_Left_vlPFC.index.tolist())

['F0final_sma', 'voicingFinalUnclipped_sma', 'shimmerLocal_sma', 'logHNR_sma', 'audspec_lengthL1norm_sma', 'pcm_RMSenergy_sma', 'pcm_zcr_sma', 'audSpec_Rfilt_sma[20]', 'audSpec_Rfilt_sma[21]', 'audSpec_Rfilt_sma[23]', 'audSpec_Rfilt_sma[24]', 'audSpec_Rfilt_sma[25]', 'pcm_fftMag_fband250-650_sma', 'pcm_fftMag_fband1000-4000_sma', 'pcm_fftMag_spectralRollOff25.0_sma', 'pcm_fftMag_spectralRollOff50.0_sma', 'pcm_fftMag_spectralRollOff75.0_sma', 'pcm_fftMag_spectralRollOff90.0_sma', 'pcm_fftMag_spectralFlux_sma', 'pcm_fftMag_spectralCentroid_sma', 'pcm_fftMag_spectralEntropy_sma', 'pcm_fftMag_spectralVariance_sma', 'pcm_fftMag_spectralSlope_sma', 'pcm_fftMag_psySharpness_sma', 'pcm_fftMag_spectralHarmonicity_sma', 'mfcc_sma[1]', 'mfcc_sma[3]', 'mfcc_sma[5]', 'mfcc_sma[6]', 'mfcc_sma[7]', 'mfcc_sma[8]', 'mfcc_sma[9]', 'mfcc_sma[11]', 'mfcc_sma[12]', 'mfcc_sma[13]', 'Loudness_sma3', 'alphaRatio_sma3', 'hammarbergIndex_sma3', 'slope0-500_sma3', 'slope500-1500_sma3', 'spectralFlux_sma3', 'mfcc

##Right OFC

In [ ]:
correlations_Right_OFC = powerAllLLDdf.corr()['Power: Right OFC'][4:]#.abs()

In [ ]:
positiveCorrelationCondition = correlations_Right_OFC > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Right_OFC < -1* rValFor95ConfidenceLevel
AcousticCorrelates95_Right_OFC = correlations_Right_OFC[ positiveCorrelationCondition | negativeCorrelationCondition]
AcousticCorrelates95_Right_OFC.head = 'r'

print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Right_OFC))
AcousticCorrelates95_Right_OFC.nlargest(len(AcousticCorrelates95_Right_OFC)) # for sorting

# of acoustic features where p < 0.05: 8


pcm_fftMag_spectralVariance_sma    0.065819
audSpec_Rfilt_sma[7]               0.060958
mfcc_sma_de[13]                    0.060899
mfcc_sma_de[3]                     0.059586
mfcc_sma_de[6].1                  -0.075108
mfcc_sma_de[6]                    -0.075324
mfcc_sma_de[7].1                  -0.097232
mfcc_sma_de[7]                    -0.104380
Name: Power: Right OFC, dtype: float64

In [ ]:
AcousticCorrelates95_Right_OFC.nlargest(len(AcousticCorrelates95_Right_OFC)).to_csv(date + '_noModeratorSpeech_AcousticCorrelates95_Right_OFC.csv',header=['r'],index_label='Acoustic Feature')

In [ ]:
print(AcousticCorrelates95_Right_OFC.index.tolist())

['audSpec_Rfilt_sma[7]', 'pcm_fftMag_spectralVariance_sma', 'mfcc_sma_de[3]', 'mfcc_sma_de[6]', 'mfcc_sma_de[7]', 'mfcc_sma_de[13]', 'mfcc_sma_de[6].1', 'mfcc_sma_de[7].1']


##Left OFC

In [ ]:
correlations_Left_OFC = powerAllLLDdf.corr()['Power: Left OFC'][8:]#.abs()

In [ ]:
positiveCorrelationCondition = correlations_Left_OFC > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Left_OFC < -1* rValFor95ConfidenceLevel
AcousticCorrelates95_Left_OFC = correlations_Left_OFC[ positiveCorrelationCondition | negativeCorrelationCondition]
AcousticCorrelates95_Left_OFC.head = 'r'

print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Left_OFC))
AcousticCorrelates95_Left_OFC.nlargest(len(AcousticCorrelates95_Left_OFC)) # for sorting

# of acoustic features where p < 0.05: 12


mfcc_sma[6]                     0.076669
mfcc_sma[6].1                   0.076366
voicingFinalUnclipped_sma_de    0.061217
mfcc_sma_de[3].1               -0.060426
mfcc_sma_de[11].1              -0.061127
mfcc_sma_de[2].1               -0.063314
mfcc_sma[3]                    -0.068217
mfcc3_sma3                     -0.068217
mfcc_sma[3].1                  -0.071831
mfcc_sma[4].1                  -0.072390
mfcc_sma[4]                    -0.078029
mfcc4_sma3                     -0.078029
Name: Power: Left OFC, dtype: float64

In [ ]:
AcousticCorrelates95_Left_OFC.nlargest(len(AcousticCorrelates95_Left_OFC)).to_csv(date + '_noModeratorSpeech_AcousticCorrelates95_Left_OFC.csv',header=['r'],index_label='Acoustic Feature')

In [ ]:
print(AcousticCorrelates95_Left_OFC.index.tolist())

['mfcc_sma[3]', 'mfcc_sma[4]', 'mfcc_sma[6]', 'mfcc3_sma3', 'mfcc4_sma3', 'mfcc_sma[3].1', 'mfcc_sma[4].1', 'mfcc_sma[6].1', 'voicingFinalUnclipped_sma_de', 'mfcc_sma_de[2].1', 'mfcc_sma_de[3].1', 'mfcc_sma_de[11].1']


#Feb 27 Experiment 1 (waiting for textgrids)

##Setup

In [ ]:
date = '2023-02-27_Experiment1_'

In [ ]:
powerAmplitudeDF = pd.read_csv('2023-02-27-Experiment1-Power-Amplitude.csv')
del powerAmplitudeDF['Unnamed: 0']
powerAmplitudeDF.head()

,Power: Left OFC,Power: Left vlPFC,Power: Right OFC,Power: Right vlPFC
0,5.686801,9.739613,8.195255,8.018249
1,9.301764,10.138419,5.699974,8.329905
2,3.541367,9.768278,4.449306,13.404476
3,8.466536,6.540455,10.163884,11.027338
4,11.272942,8.775278,9.911861,12.002101


In [ ]:
powerAmplitudeDF.shape

(414, 4)

In [ ]:
acousticFeatsDF = pd.read_csv('AcousticFeatures_noModeratorSpeech_aDBS012_' + '2023-02-27_Experiment1_' + 'audio_amplitude.csv')
del acousticFeatsDF['Unnamed: 0']
acousticFeatsDF.head()

,F0final_sma,voicingFinalUnclipped_sma,jitterLocal_sma,jitterDDP_sma,shimmerLocal_sma,logHNR_sma,audspec_lengthL1norm_sma,audspecRasta_lengthL1norm_sma,pcm_RMSenergy_sma,pcm_zcr_sma,...,lspFreq_sma_de[2],lspFreq_sma_de[3],lspFreq_sma_de[4],lspFreq_sma_de[5],lspFreq_sma_de[6],lspFreq_sma_de[7],pcm_zcr_sma_de.1,voiceProb_sma_de,F0_sma_de,F0env_sma_de
0,0.0,0.0,0.0,0.0,0.0,-100.0,0.001034,0.988341,0.0,0.999244,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,-100.0,0.001034,0.988341,0.0,0.999244,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,-100.0,0.001034,0.988341,0.0,0.999244,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,-100.0,0.001034,0.988341,0.0,0.999244,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,-100.0,0.001034,0.988341,0.0,0.999244,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
powerAmplitudeDF = powerAmplitudeDF.iloc[:acousticFeatsDF.shape[0]]
powerAmplitudeDF.shape

(381, 4)

In [ ]:
powerAmplitudeAllLLDdf = pd.concat([powerAmplitudeDF,acousticFeatsDF],axis=1)
powerAmplitudeAllLLDdf.shape

(381, 211)

Website to compute p value from r value and n: http://vassarstats.net/tabs_r.html

In [ ]:
rValFor99ConfidenceLevel = 0.132
rValFor95ConfidenceLevel = 0.101

##random noise t-test (10)

In [ ]:
import numpy as np

In [ ]:
# Set a seed for reproducibility (optional)
np.random.seed(33)

# Mean and standard deviation of the Gaussian distribution
mean_value = 0
std_dev = 1

# Generate a random element numpy array from a Gaussian distribution
random_array = np.random.normal(mean_value, std_dev, powerAmplitudeAllLLDdf.shape[0])


In [ ]:
powerAmplitudeAllLLDdf['Noise'] = random_array

In [ ]:
powerAmplitudeAllLLDdf.corr()['Noise'][4:-1]#.abs()

F0final_sma                 -0.079760
voicingFinalUnclipped_sma   -0.091099
jitterLocal_sma             -0.065737
jitterDDP_sma               -0.080899
shimmerLocal_sma            -0.085380
                               ...   
lspFreq_sma_de[7]           -0.102589
pcm_zcr_sma_de.1             0.049295
voiceProb_sma_de            -0.013974
F0_sma_de                    0.042997
F0env_sma_de                -0.068461
Name: Noise, Length: 207, dtype: float64

In [ ]:
correlations_Noise = powerAmplitudeAllLLDdf.corr()['Noise'][4:-1]#.abs()

In [ ]:
positiveCorrelationCondition = correlations_Noise > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Noise < -1* rValFor95ConfidenceLevel
noiseCorrelates = correlations_Noise[ positiveCorrelationCondition | negativeCorrelationCondition]
noiseCorrelates.head = 'r'

print("# of acoustic features where p < 0.05:", len(noiseCorrelates))
noiseCorrelates.nlargest(len(noiseCorrelates)) # for sorting

# of acoustic features where p < 0.05: 10


F0env_sma                          0.130533
mfcc_sma[10]                       0.121059
mfcc_sma[10].1                     0.114129
slope500-1500_sma3                 0.108236
mfcc_sma[6]                        0.103016
pcm_RMSenergy_sma                 -0.102180
lspFreq_sma_de[7]                 -0.102589
logHNR_sma                        -0.104116
pcm_fftMag_spectralKurtosis_sma   -0.104781
jitterLocal_sma3nz                -0.109208
Name: Noise, dtype: float64

##Right vLPFC

In [ ]:
correlations_Right_vlPFC = powerAmplitudeAllLLDdf.corr()['Power: Right vlPFC'][4:]#.abs()

In [ ]:
positiveCorrelationCondition = correlations_Right_vlPFC > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Right_vlPFC < -1* rValFor95ConfidenceLevel
AcousticCorrelates95_Right_vlPFC = correlations_Right_vlPFC[ positiveCorrelationCondition | negativeCorrelationCondition]
AcousticCorrelates95_Right_vlPFC.head = 'r'

print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Right_vlPFC))
AcousticCorrelates95_Right_vlPFC.nlargest(len(AcousticCorrelates95_Right_vlPFC)) # for sorting

# of acoustic features where p < 0.05: 2


mfcc_sma_de[12].1   -0.121493
mfcc_sma_de[12]     -0.133622
Name: Power: Right vlPFC, dtype: float64

In [ ]:
AcousticCorrelates95_Right_vlPFC.nlargest(len(AcousticCorrelates95_Right_vlPFC)).to_csv(date + '_AcousticCorrelates95_Right_vlPFC.csv',header=['r'],index_label='Acoustic Feature')

In [ ]:
print(AcousticCorrelates95_Right_vlPFC.index.tolist())

['mfcc_sma_de[12]', 'mfcc_sma_de[12].1']


##Left vLPFC

In [ ]:
correlations_Left_vlPFC = powerAmplitudeAllLLDdf.corr()['Power: Left vlPFC'][4:]#.abs()
correlations_Left_vlPFC

F0final_sma                  0.068850
voicingFinalUnclipped_sma    0.070819
jitterLocal_sma              0.109990
jitterDDP_sma                0.102479
shimmerLocal_sma             0.086703
                               ...   
lspFreq_sma_de[7]           -0.002623
pcm_zcr_sma_de.1             0.004705
voiceProb_sma_de            -0.000086
F0_sma_de                    0.011111
F0env_sma_de                -0.024345
Name: Power: Left vlPFC, Length: 207, dtype: float64

In [ ]:
positiveCorrelationCondition = correlations_Left_vlPFC > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Left_vlPFC < -1* rValFor95ConfidenceLevel
AcousticCorrelates95_Left_vlPFC = correlations_Left_vlPFC[ positiveCorrelationCondition | negativeCorrelationCondition]
AcousticCorrelates95_Left_vlPFC.head = 'r'

print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Left_vlPFC))
AcousticCorrelates95_Left_vlPFC.nlargest(len(AcousticCorrelates95_Left_vlPFC)) # for sorting

# of acoustic features where p < 0.05: 3


jitterLocal_sma        0.109990
jitterDDP_sma          0.102479
shimmerLocal_sma_de   -0.110014
Name: Power: Left vlPFC, dtype: float64

In [ ]:
AcousticCorrelates95_Left_vlPFC.nlargest(len(AcousticCorrelates95_Left_vlPFC)).to_csv(date + '_AcousticCorrelates95_Left_vlPFC.csv',header=['r'],index_label='Acoustic Feature')

In [ ]:
print(AcousticCorrelates95_Left_vlPFC.index.tolist())

['jitterLocal_sma', 'jitterDDP_sma', 'shimmerLocal_sma_de']


##Right OFC

In [ ]:
correlations_Right_OFC = powerAmplitudeAllLLDdf.corr()['Power: Right OFC'][4:]#.abs()


In [ ]:
positiveCorrelationCondition = correlations_Right_OFC > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Right_OFC < -1* rValFor95ConfidenceLevel
AcousticCorrelates95_Right_OFC = correlations_Right_OFC[ positiveCorrelationCondition | negativeCorrelationCondition]
AcousticCorrelates95_Right_OFC.head = 'r'

print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Right_OFC))
AcousticCorrelates95_Right_OFC.nlargest(len(AcousticCorrelates95_Right_OFC)) # for sorting

# of acoustic features where p < 0.05: 1


shimmerLocal_sma_de    0.102154
Name: Power: Right OFC, dtype: float64

In [ ]:
AcousticCorrelates95_Right_OFC.nlargest(len(AcousticCorrelates95_Right_OFC)).to_csv(date + '_AcousticCorrelates95_Right_OFC.csv',header=['r'],index_label='Acoustic Feature')

In [ ]:
print(AcousticCorrelates95_Right_OFC.index.tolist())

['shimmerLocal_sma_de']


##Left OFC

In [ ]:
correlations_Left_OFC = powerAmplitudeAllLLDdf.corr()['Power: Left OFC'][8:]#.abs()


In [ ]:
positiveCorrelationCondition = correlations_Left_OFC > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Left_OFC < -1* rValFor95ConfidenceLevel
AcousticCorrelates95_Left_OFC = correlations_Left_OFC[ positiveCorrelationCondition | negativeCorrelationCondition]
AcousticCorrelates95_Left_OFC.head = 'r'

print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Left_OFC))
AcousticCorrelates95_Left_OFC.nlargest(len(AcousticCorrelates95_Left_OFC)) # for sorting

# of acoustic features where p < 0.05: 3


mfcc_sma[2].1    0.135997
mfcc_sma[2]      0.107883
mfcc2_sma3       0.107883
Name: Power: Left OFC, dtype: float64

In [ ]:
AcousticCorrelates95_Left_OFC.nlargest(len(AcousticCorrelates95_Left_OFC)).to_csv(date + '_AcousticCorrelates95_Left_OFC.csv',header=['r'],index_label='Acoustic Feature')

In [ ]:
print(AcousticCorrelates95_Left_OFC.index.tolist())

['mfcc_sma[2]', 'mfcc2_sma3', 'mfcc_sma[2].1']


#Feb 27 Experiment 5 (waiting for textgrids)

##Setup

In [ ]:
date ='2023-02-27_Experiment5'

In [ ]:
powerAmplitudeDF = pd.read_csv('2023-02-27-Experiment5-Power.csv')
del powerAmplitudeDF['Unnamed: 0']
powerAmplitudeDF.head()

,Power: Left OFC,Power: Left vlPFC,Power: Right OFC,Power: Right vlPFC
0,6.040388,9.369975,11.474579,9.765721
1,4.740961,11.204577,9.600362,9.777031
2,4.904171,8.815254,4.404419,11.895500
3,5.530016,7.770287,8.693739,10.962857
4,6.540160,11.214869,5.332976,10.099922


In [ ]:
powerAmplitudeDF.shape

(624, 4)

In [ ]:
acousticFeatsDF = pd.read_csv('AcousticFeatures_noModeratorSpeech_aDBS012_' + '2023-02-27_Experiment5_' + 'audio_amplitude.csv')
del acousticFeatsDF['Unnamed: 0']
acousticFeatsDF.head()

,F0final_sma,voicingFinalUnclipped_sma,jitterLocal_sma,jitterDDP_sma,shimmerLocal_sma,logHNR_sma,audspec_lengthL1norm_sma,audspecRasta_lengthL1norm_sma,pcm_RMSenergy_sma,pcm_zcr_sma,...,lspFreq_sma_de[2],lspFreq_sma_de[3],lspFreq_sma_de[4],lspFreq_sma_de[5],lspFreq_sma_de[6],lspFreq_sma_de[7],pcm_zcr_sma_de.1,voiceProb_sma_de,F0_sma_de,F0env_sma_de
0,0.000000,0.000000,0.000000,0.000000,0.000000,-100.00000,0.001034,0.988341,0.000000,0.999244,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,-99.65611,0.005660,2.070137,0.000039,0.951934,...,0.000020,-0.000762,-0.000765,-0.002072,-0.002411,-0.004873,-0.009227,0.003475,0.000000,0.000000
2,5.074933,0.088293,0.003110,0.000000,0.020596,-92.11421,0.095801,5.875671,0.001101,0.309874,...,0.000129,0.000296,-0.000063,0.000073,-0.000141,-0.000285,-0.000306,0.000336,0.000000,1.862618
3,21.654139,0.264973,0.017881,0.017846,0.073430,-66.56270,0.105928,3.049266,0.001712,0.344805,...,0.000262,0.000287,-0.000472,-0.000585,-0.000327,-0.000235,-0.000139,0.002086,1.497543,1.848263
4,14.408586,0.208845,0.002083,0.001160,0.013484,-74.86962,0.133912,1.252230,0.001552,0.212804,...,-0.000041,-0.000360,0.000078,0.000143,0.000336,0.000222,0.000155,-0.000961,-1.497543,0.147647


In [ ]:
acousticFeatsDF.shape

(365, 207)

Sameer: all power values after t=372 is NaN anyways, explains large size dispairtiy

In [ ]:
powerAmplitudeDF = powerAmplitudeDF.iloc[:acousticFeatsDF.shape[0]]
powerAmplitudeDF.shape

(365, 4)

In [ ]:
powerAmplitudeAllLLDdf = pd.concat([powerAmplitudeDF,acousticFeatsDF],axis=1)
powerAmplitudeAllLLDdf.shape

(365, 211)

Website to compute p value from r value and n: http://vassarstats.net/tabs_r.html

Need r >= 0.077 for p < 0.01
Need r >= 0.059 for p < 0.05

In [ ]:
rValFor99ConfidenceLevel = 0.134
rValFor95ConfidenceLevel = 0.102

##random noise t-test (9)

In [ ]:
import numpy as np

In [ ]:
# Set a seed for reproducibility (optional)
np.random.seed(33)

# Mean and standard deviation of the Gaussian distribution
mean_value = 0
std_dev = 1

# Generate a random element numpy array from a Gaussian distribution
random_array = np.random.normal(mean_value, std_dev, powerAmplitudeAllLLDdf.shape[0])


In [ ]:
powerAmplitudeAllLLDdf['Noise'] = random_array

In [ ]:
powerAmplitudeAllLLDdf.corr()['Noise'][4:-1]#.abs()

F0final_sma                 -0.071797
voicingFinalUnclipped_sma   -0.076833
jitterLocal_sma             -0.014940
jitterDDP_sma               -0.010610
shimmerLocal_sma            -0.051291
                               ...   
lspFreq_sma_de[7]            0.041897
pcm_zcr_sma_de.1            -0.041198
voiceProb_sma_de             0.009558
F0_sma_de                    0.012827
F0env_sma_de                -0.006773
Name: Noise, Length: 207, dtype: float64

In [ ]:
correlations_Noise = powerAmplitudeAllLLDdf.corr()['Noise'][4:-1]#.abs()

In [ ]:
positiveCorrelationCondition = correlations_Noise > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Noise < -1* rValFor95ConfidenceLevel
noiseCorrelates = correlations_Noise[ positiveCorrelationCondition | negativeCorrelationCondition]
noiseCorrelates.head = 'r'

print("# of acoustic features where p < 0.05:", len(noiseCorrelates))
noiseCorrelates.nlargest(len(noiseCorrelates)) # for sorting

# of acoustic features where p < 0.05: 9


mfcc_sma_de[1]              0.109871
mfcc_sma_de[1].1            0.104717
mfcc_sma_de[4]              0.102487
audSpec_Rfilt_sma_de[17]   -0.103444
audSpec_Rfilt_sma_de[18]   -0.112165
mfcc_sma_de[12].1          -0.145052
mfcc_sma_de[11]            -0.146326
mfcc_sma_de[11].1          -0.155130
mfcc_sma_de[12]            -0.158149
Name: Noise, dtype: float64

##Right vLPFC

In [ ]:
correlations_Right_vlPFC = powerAmplitudeAllLLDdf.corr()['Power: Right vlPFC'][4:]#.abs()

In [ ]:
positiveCorrelationCondition = correlations_Right_vlPFC > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Right_vlPFC < -1* rValFor95ConfidenceLevel
AcousticCorrelates95_Right_vlPFC = correlations_Right_vlPFC[ positiveCorrelationCondition | negativeCorrelationCondition]
AcousticCorrelates95_Right_vlPFC.head = 'r'

print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Right_vlPFC))
AcousticCorrelates95_Right_vlPFC.nlargest(len(AcousticCorrelates95_Right_vlPFC)) # for sorting

# of acoustic features where p < 0.05: 10


mfcc_sma[12].1                        0.130416
pcm_fftMag_spectralSlope_sma          0.129381
mfcc_sma[12]                          0.126649
spectralFlux_sma3                    -0.103421
pcm_fftMag_spectralFlux_sma          -0.104770
mfcc_sma_de[10]                      -0.106423
pcm_fftMag_spectralKurtosis_sma_de   -0.122354
pcm_intensity_sma                    -0.123894
pcm_fftMag_fband250-650_sma          -0.125326
pcm_fftMag_spectralHarmonicity_sma   -0.128665
Name: Power: Right vlPFC, dtype: float64

In [ ]:
AcousticCorrelates95_Right_vlPFC.nlargest(len(AcousticCorrelates95_Right_vlPFC)).to_csv(date + '_AcousticCorrelates95_Right_vlPFC.csv',header=['r'],index_label='Acoustic Feature')

In [ ]:
print(AcousticCorrelates95_Right_vlPFC.index.tolist())

['pcm_fftMag_fband250-650_sma', 'pcm_fftMag_spectralFlux_sma', 'pcm_fftMag_spectralSlope_sma', 'pcm_fftMag_spectralHarmonicity_sma', 'mfcc_sma[12]', 'spectralFlux_sma3', 'pcm_intensity_sma', 'mfcc_sma[12].1', 'pcm_fftMag_spectralKurtosis_sma_de', 'mfcc_sma_de[10]']


##Left vLPFC

In [ ]:
correlations_Left_vlPFC = powerAmplitudeAllLLDdf.corr()['Power: Left vlPFC'][4:]#.abs()
correlations_Left_vlPFC

F0final_sma                  0.127205
voicingFinalUnclipped_sma    0.139767
jitterLocal_sma              0.091145
jitterDDP_sma                0.130096
shimmerLocal_sma             0.123459
                               ...   
lspFreq_sma_de[7]           -0.028354
pcm_zcr_sma_de.1            -0.040467
voiceProb_sma_de             0.031438
F0_sma_de                    0.037576
F0env_sma_de                 0.019421
Name: Power: Left vlPFC, Length: 207, dtype: float64

In [ ]:
positiveCorrelationCondition = correlations_Left_vlPFC > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Left_vlPFC < -1* rValFor95ConfidenceLevel
AcousticCorrelates95_Left_vlPFC = correlations_Left_vlPFC[ positiveCorrelationCondition | negativeCorrelationCondition]
AcousticCorrelates95_Left_vlPFC.head = 'r'

print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Left_vlPFC))
AcousticCorrelates95_Left_vlPFC.nlargest(len(AcousticCorrelates95_Left_vlPFC)) # for sorting

# of acoustic features where p < 0.05: 83


hammarbergIndex_sma3     0.165189
audSpec_Rfilt_sma[10]    0.163511
audSpec_Rfilt_sma[11]    0.162552
audSpec_Rfilt_sma[7]     0.159103
F3bandwidth_sma3nz       0.155206
                           ...   
slope500-1500_sma3      -0.149682
lspFreq_sma[7]          -0.149824
lspFreq_sma[6]          -0.153129
alphaRatio_sma3         -0.160347
HNRdBACF_sma3nz         -0.178553
Name: Power: Left vlPFC, Length: 83, dtype: float64

In [ ]:
AcousticCorrelates95_Left_vlPFC.nlargest(len(AcousticCorrelates95_Left_vlPFC)).to_csv(date + '_AcousticCorrelates95_Left_vlPFC.csv',header=['r'],index_label='Acoustic Feature')

In [ ]:
print(AcousticCorrelates95_Left_vlPFC.index.tolist())

['F0final_sma', 'voicingFinalUnclipped_sma', 'jitterDDP_sma', 'shimmerLocal_sma', 'logHNR_sma', 'audspec_lengthL1norm_sma', 'audspecRasta_lengthL1norm_sma', 'pcm_RMSenergy_sma', 'pcm_zcr_sma', 'audSpec_Rfilt_sma[0]', 'audSpec_Rfilt_sma[1]', 'audSpec_Rfilt_sma[3]', 'audSpec_Rfilt_sma[4]', 'audSpec_Rfilt_sma[5]', 'audSpec_Rfilt_sma[6]', 'audSpec_Rfilt_sma[7]', 'audSpec_Rfilt_sma[8]', 'audSpec_Rfilt_sma[9]', 'audSpec_Rfilt_sma[10]', 'audSpec_Rfilt_sma[11]', 'audSpec_Rfilt_sma[12]', 'audSpec_Rfilt_sma[13]', 'audSpec_Rfilt_sma[14]', 'audSpec_Rfilt_sma[15]', 'audSpec_Rfilt_sma[16]', 'audSpec_Rfilt_sma[17]', 'audSpec_Rfilt_sma[18]', 'audSpec_Rfilt_sma[19]', 'audSpec_Rfilt_sma[20]', 'audSpec_Rfilt_sma[21]', 'audSpec_Rfilt_sma[22]', 'audSpec_Rfilt_sma[23]', 'audSpec_Rfilt_sma[24]', 'audSpec_Rfilt_sma[25]', 'pcm_fftMag_fband250-650_sma', 'pcm_fftMag_spectralRollOff25.0_sma', 'pcm_fftMag_spectralRollOff50.0_sma', 'pcm_fftMag_spectralRollOff75.0_sma', 'pcm_fftMag_spectralRollOff90.0_sma', 'pcm_fft

##Right OFC

In [ ]:
correlations_Right_OFC = powerAmplitudeAllLLDdf.corr()['Power: Right OFC'][4:]#.abs()


In [ ]:
positiveCorrelationCondition = correlations_Right_OFC > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Right_OFC < -1* rValFor95ConfidenceLevel
AcousticCorrelates95_Right_OFC = correlations_Right_OFC[ positiveCorrelationCondition | negativeCorrelationCondition]
AcousticCorrelates95_Right_OFC.head = 'r'

print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Right_OFC))
AcousticCorrelates95_Right_OFC.nlargest(len(AcousticCorrelates95_Right_OFC)) # for sorting

# of acoustic features where p < 0.05: 2


mfcc_sma_de[5]                       -0.106432
pcm_fftMag_spectralVariance_sma_de   -0.131466
Name: Power: Right OFC, dtype: float64

In [ ]:
AcousticCorrelates95_Right_OFC.nlargest(len(AcousticCorrelates95_Right_OFC)).to_csv(date + '_AcousticCorrelates95_Right_OFC.csv',header=['r'],index_label='Acoustic Feature')

In [ ]:
print(AcousticCorrelates95_Right_OFC.index.tolist())

['pcm_fftMag_spectralVariance_sma_de', 'mfcc_sma_de[5]']


##Left OFC

In [ ]:
correlations_Left_OFC = powerAmplitudeAllLLDdf.corr()['Power: Left OFC'][8:]#.abs()

In [ ]:
positiveCorrelationCondition = correlations_Left_OFC > rValFor95ConfidenceLevel
negativeCorrelationCondition = correlations_Left_OFC < -1* rValFor95ConfidenceLevel
AcousticCorrelates95_Left_OFC = correlations_Left_OFC[ positiveCorrelationCondition | negativeCorrelationCondition]
AcousticCorrelates95_Left_OFC.head = 'r'

print("# of acoustic features where p < 0.05:", len(AcousticCorrelates95_Left_OFC))
AcousticCorrelates95_Left_OFC.nlargest(len(AcousticCorrelates95_Left_OFC)) # for sorting

# of acoustic features where p < 0.05: 32


pcm_RMSenergy_sma                     0.155620
pcm_fftMag_spectralFlux_sma           0.155256
spectralFlux_sma3                     0.153749
pcm_fftMag_fband250-650_sma           0.146709
shimmerLocaldB_sma3nz                 0.145798
pcm_intensity_sma                     0.144588
pcm_fftMag_spectralHarmonicity_sma    0.143261
Loudness_sma3                         0.137988
audspec_lengthL1norm_sma              0.137988
pcm_loudness_sma                      0.135379
F0semitoneFrom27.5Hz_sma3nz           0.134802
F1amplitudeLogRelF0_sma3nz            0.130531
F3amplitudeLogRelF0_sma3nz            0.130439
F2amplitudeLogRelF0_sma3nz            0.129831
shimmerLocal_sma                      0.123066
logHNR_sma                            0.115550
F0_sma                                0.113753
pcm_fftMag_fband1000-4000_sma         0.104637
lspFreq_sma[0]                       -0.103585
lspFreq_sma[3]                       -0.103999
mfcc_sma[5]                          -0.104403
mfcc_sma[13] 

In [ ]:
date

'2023-02-27_Experiment5'

In [ ]:
AcousticCorrelates95_Left_OFC.nlargest(len(AcousticCorrelates95_Left_OFC)).to_csv(date + '_AcousticCorrelates95_Left_OFC.csv',header=['r'],index_label='Acoustic Feature')

In [ ]:
print(AcousticCorrelates95_Left_OFC.index.tolist())

['shimmerLocal_sma', 'logHNR_sma', 'audspec_lengthL1norm_sma', 'pcm_RMSenergy_sma', 'pcm_fftMag_fband250-650_sma', 'pcm_fftMag_fband1000-4000_sma', 'pcm_fftMag_spectralFlux_sma', 'pcm_fftMag_spectralSlope_sma', 'pcm_fftMag_spectralHarmonicity_sma', 'mfcc_sma[5]', 'mfcc_sma[6]', 'mfcc_sma[7]', 'mfcc_sma[10]', 'mfcc_sma[13]', 'Loudness_sma3', 'spectralFlux_sma3', 'F0semitoneFrom27.5Hz_sma3nz', 'shimmerLocaldB_sma3nz', 'F1amplitudeLogRelF0_sma3nz', 'F2amplitudeLogRelF0_sma3nz', 'F3amplitudeLogRelF0_sma3nz', 'pcm_intensity_sma', 'pcm_loudness_sma', 'mfcc_sma[5].1', 'mfcc_sma[6].1', 'mfcc_sma[7].1', 'mfcc_sma[10].1', 'lspFreq_sma[0]', 'lspFreq_sma[1]', 'lspFreq_sma[2]', 'lspFreq_sma[3]', 'F0_sma']
